# Valuation using Inferred FX Rates (for an inline portfolio)

This notebook shows you how to value an inline portfolio using recipes with FX rates inferred and not explicitly loaded into the quotes store. The notebook shows the following workflow:

- We run a valuation on an inline portfolio which has a base currency of GBP
- In the same portfolio, LUSID values the JPY instruments by inverting the provided JPY/GBP FX rate.
- LUSID then values the AUD instruments by triangulating the provided GBP/USD and AUD/USD FX rates.

For some background, an inline portfolio is not a "real" portfolio in LUSID per se. Rather the inline portfolio holdings (instruments and weights) are defined in the inline valuation request. This allows users to run ad-hoc valuations on a set of weighted instruments without requiring a full portfolio setup.

## 1. Setup LUSID

In [1]:
# Import lusid specific packages
# These are the core lusid packages for interacting with the API via Python
import lusid
import lusid.models as models
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame

import os
import pandas as pd
import pytz
from datetime import datetime

pd.set_option("display.max_columns", None)

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

/opt/conda/lib/python3.9/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
# define the apis
aggregation_api = api_factory.build(lusid.AggregationApi)
quotes_api = api_factory.build(lusid.QuotesApi)
configuration_recipe_api = api_factory.build(lusid.api.ConfigurationRecipeApi)

In [3]:
# Defining variables
valuation_date = datetime(year=2022, month=3, day=8, tzinfo=pytz.UTC).isoformat()
scope = "Ibor"
portfolio_code = "FX_Conversion_NB"

## 2. Load Quotes

### 2.1 Instrument Value Quotes

Load 5 prices into the quotes store with associated instrument 'ClientInternal' identifiers. These will be linked to the weighted instruments created below in Section 3.

In [4]:
values = [20, 50, 100, 75, 60]

for i in range(1,6):
    quotes_api.upsert_quotes(
        scope = scope,
        request_body = {f"quote_{i}": models.UpsertQuoteRequest(
            quote_id=models.QuoteId(
                models.QuoteSeriesId(
                    provider="Lusid",
                    instrument_id=f"client_internal_{i}",
                    instrument_id_type="ClientInternal",
                    quote_type="Price",
                    field="mid"
                
                ),
                effective_at = valuation_date
            ),
            metric_value=models.MetricValue(
                value=values[i-1],
                unit="GBP"
            )
        )}
        
    )

### 2.2 FX Rate Quotes

Load FX rates required for FX rate inference into the quotes store.

In [5]:
def spot_request(from_ccy, to_ccy, rate, valuation_date):
            return models.UpsertQuoteRequest(
                       quote_id=models.QuoteId(
                           models.QuoteSeriesId(
                               provider='Lusid',
                               instrument_id=f'{from_ccy}/{to_ccy}',
                               instrument_id_type='CurrencyPair',
                               quote_type='Rate',
                               field='mid'
                           ),
                           effective_at=valuation_date
                       ),
                       metric_value=models.MetricValue(
                           value=rate,
                           unit=f'{from_ccy}/{to_ccy}'
                       ),
                       lineage='None'
            )

#### JPY/GBP FX Rate Quote

Load JPY/GBP FX rate into the quotes store for the valuation date.

In [6]:
response = quotes_api.upsert_quotes(scope=scope,
                                   request_body={"1": spot_request("JPY", "GBP", 0.006618, valuation_date)})

#### GBP/USD FX Rate Quote

Load GBP/USD FX rate into the quotes store for the valuation date.

In [7]:
response = quotes_api.upsert_quotes(scope=scope,
                                   request_body={"1": spot_request("GBP", "USD", 1.3106, valuation_date)})

#### AUD/USD FX Rate Quote

Load AUD/USD FX rate into the quotes store for the valuation date.

In [8]:
response = quotes_api.upsert_quotes(scope=scope,
                                   request_body={"1": spot_request("AUD", "USD", 0.7319, valuation_date)})

## 3. Define Weighted Instruments

Define 5 weighted equity instruments with the same 'ClientInternal' identifiers as those of the 5 quotes, a quantity of 1 and a currency of GBP (same as for the quotes). Put these 5 definitions into a list to be passed into the inline valuations request in Section 4.

In [9]:
weighted_instruments = []

for i in range(1, 6):
    weighted_instrument = lusid.WeightedInstrument(
        quantity=1,
        holding_identifier=f"client_internal_{i}",
        instrument=models.Equity(
            identifiers= lusid.EquityAllOfIdentifiers(
                client_internal=f"client_internal_{i}",
            ),
            dom_ccy="GBP",
            instrument_type="Equity",
        ), 
    )
        
    
    weighted_instruments.append(weighted_instrument)

## 4. Create Valuation Recipe

Define a valuation recipe to use the quotes loaded into the quotes store. 

*** Important *** - The attempt_to_infer_missing_fx option is set to True on line 36 below enabling LUSID to attempt to determine missing FX rates from rates in the quotes store.

In [10]:
# Create recipes
recipe_scope="FX_Conversion_NB"
recipe_code="FX_Conversion_NB"


# Create a recipe to perform a valuation
configuration_recipe = models.ConfigurationRecipe(
    scope=recipe_scope,
    code=recipe_code,
    market=models.MarketContext(
        market_rules=[
            # define how to resolve the quotes
            models.MarketDataKeyRule(
                key="Equity.ClientInternal.*",
                supplier="Lusid",
                data_scope=scope,
                quote_type="Price",
                field="mid",
            ),
            models.MarketDataKeyRule(
                key='Fx.CurrencyPair.*',
                data_scope=scope,
                supplier='Lusid',
                quote_type='Rate',
                quote_interval='1D.0D',
                field="mid"
            )
        ],
        options=models.MarketOptions(
            default_supplier="Lusid",
            default_instrument_code_type="Isin",
            default_scope='Lusid',
        ### IMPORTANT ###
        # This enables FX rate inference
        #------------------------------------
            attempt_to_infer_missing_fx=True,
#             save_inferred_missing_fx=True,
        #------------------------------------
        ),
    ),
    pricing=models.PricingContext(
        options={"AllowPartiallySuccessfulEvaluation": True},
    ),
)

upsert_configuration_recipe_response = configuration_recipe_api.upsert_configuration_recipe(
    upsert_recipe_request=models.UpsertRecipeRequest(
        configuration_recipe=configuration_recipe
    )
)

## 5. Run Inline Valuation Requests

Create valuation requests for the predefined weighted instruments with JPY & AUD report currencies (no explicit FX rates from GBP to these in the quotes store). 

### 5.1 Define Inline Valuation Request Function

In [11]:
def generate_valuation_request(valuation_effectiveAt, report_currency, instruments):

    # Create the valuation request
    valuation_request = models.InlineValuationRequest(
        recipe_id=models.ResourceId(
            scope=recipe_scope, code=recipe_code
        ),
        metrics=[
            models.AggregateSpec("Valuation/PvInReportCcy", "Value"),
            models.AggregateSpec("Valuation/PvInReportCcy/Ccy", "Value"),
            models.AggregateSpec("Analytic/default/InstrumentTag", "Value"),
            models.AggregateSpec("Quotes/FxRate/DomReport", "Value"),
            models.AggregateSpec("Quotes/Price", "Value"),
            models.AggregateSpec("Quotes/Price/Ccy", "Value")
        ],
        report_currency = report_currency,
        valuation_schedule=models.ValuationSchedule(
            effective_at=valuation_effectiveAt
        ),
        instruments=instruments
    )

    return valuation_request

### 5.2 Run Inline Valuation in JPY (FX Rate Inversion)

The explicit GBP/JPY FX rate does not exist in the quotes store and so LUSID inverts the JPY/GBP FX rate via the API call. This functionality is enabled by setting "attempt_to_infer_missing_fx" to true in the valuation recipe (see above).

In [12]:
aggregation = aggregation_api.get_valuation_of_weighted_instruments(
    inline_valuation_request=generate_valuation_request(
        valuation_date, "JPY", weighted_instruments
    )
)

output = pd.DataFrame(aggregation.data)
output

,Valuation/PvInReportCcy/Ccy,Valuation/PvInReportCcy,Analytic/default/InstrumentTag,Quotes/FxRate/DomReport,Quotes/Price,Quotes/Price/Ccy
0,JPY,3022.061046,client_internal_1,151.103052,20.0,GBP
1,JPY,7555.152614,client_internal_2,151.103052,50.0,GBP
2,JPY,15110.305228,client_internal_3,151.103052,100.0,GBP
3,JPY,11332.728921,client_internal_4,151.103052,75.0,GBP
4,JPY,9066.183137,client_internal_5,151.103052,60.0,GBP


### 5.3 Run Inline Valuation in AUD (FX Rate Triangulation)

The explicit GBP/AUD FX rate does not exist in the quotes store and so LUSID triangulates it from the GBP/USD and AUD/USD FX rates. This functionality is enabled by setting "attempt_to_infer_missing_fx" to true in the valuation recipe (see above).

In [13]:
aggregation = aggregation_api.get_valuation_of_weighted_instruments(
    inline_valuation_request=generate_valuation_request(
        valuation_date, "AUD", weighted_instruments
    )
)

output = pd.DataFrame(aggregation.data)
output

,Valuation/PvInReportCcy/Ccy,Valuation/PvInReportCcy,Analytic/default/InstrumentTag,Quotes/FxRate/DomReport,Quotes/Price,Quotes/Price/Ccy
0,AUD,35.813636,client_internal_1,1.790682,20.0,GBP
1,AUD,89.534089,client_internal_2,1.790682,50.0,GBP
2,AUD,179.068179,client_internal_3,1.790682,100.0,GBP
3,AUD,134.301134,client_internal_4,1.790682,75.0,GBP
4,AUD,107.440907,client_internal_5,1.790682,60.0,GBP
